# Analyzing AAL Sentiment Using FinBERT

In this notebook, I will be analyzing the different sections of AAL's 10K reports throughout the years, and brainstorm ideas on how I will score companies and create a signal. 

To get a sentiment score, I will be using an NLP model called FinBERT. You can read more about FinBERT here: https://github.com/ProsusAI/finBERT

To fit the model, you will need to download the pre-trained and fine-tuned weights from the following link and unzip to the working directory: https://gohkust-my.sharepoint.com/:u:/g/personal/imyiyang_ust_hk/EQJGiEOkhIlBqlW63TbKA3gBCYgDDcHlBCB7VTXIUMmyiA

**Note that I did not include the analyst_tone folder in my repository, this is because it's simply too large to include, so you will need to download it yourself**

# Toy Example

First, I will be borrowing code directly from a GitHub that I had used previously to also work with FinBERT. I recall that I had some issues on that machine, so I want to run a toy example before looking at AAL. You can find exactly where I get the code from here:

https://github.com/yya518/FinBERT/blob/master/FinBert%20Model%20Example.ipynb

In [32]:
import numpy as np 
import pandas as pd
import torch
import torch.nn.functional as F
from pytorch_pretrained_bert import BertTokenizer
from FinBERT.FinBERT_master.bertModel import BertClassification

In [33]:
labels = {0:'neutral', 1:'positive',2:'negative'}
num_labels= len(labels)
vocab = "finance-uncased"
vocab_path = 'FinBERT/analyst_tone/vocab'
pretrained_weights_path = "FinBERT/analyst_tone/pretrained_weights" # this is pre-trained FinBERT weights
fine_tuned_weight_path = "FinBERT/analyst_tone/fine_tuned.pth"      # this is fine-tuned FinBERT weights
max_seq_length=512
device= torch.device('cpu')

In [34]:
model = BertClassification(weight_path= pretrained_weights_path, num_labels=num_labels, vocab=vocab)

In [35]:
model.load_state_dict(torch.load(fine_tuned_weight_path, map_location='cpu'))
model.to(device)

BertClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )


In [36]:
sentences = ["there is a shortage of capital, and we need extra financing", 
             "growth is strong and we have plenty of liquidity", 
             "there are doubts about our finances", 
             "profits are flat"]

In [37]:
tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = True, do_basic_tokenize = True)

In [ ]:
model.eval()
for sent in sentences: 
    tokenized_sent = tokenizer.tokenize(sent)
    if len(tokenized_sent) > max_seq_length:
        tokenized_sent = tokenized_sent[:max_seq_length]
    
    ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
    mask_input = [1]*len(ids_review)        
    padding = [0] * (max_seq_length - len(ids_review))
    ids_review += padding
    mask_input += padding
    input_type = [0]*max_seq_length
    
    input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
    attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
    token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)
    
    with torch.set_grad_enabled(False):
        outputs = model(input_ids, token_type_ids, attention_mask)
        outputs = F.softmax(outputs,dim=1)
        print(sent, '\nFinBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')


there is a shortage of capital, and we need extra financing 
FinBERT predicted sentiment:  negative 

